In [1]:
import pandas as pd
import mlflow
from catboost import CatBoostRegressor
import yaml
import os
import joblib
import json
import pickle

In [2]:
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000


mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

EXPERIMENT_NAME = "sprint1_model_price"
RUN_NAME = "save_baseline"
REGISTRY_MODEL_NAME = "model_price_appartment"


In [3]:
with open('cv_res.json', 'r', encoding='utf-8') as file:
    metrics = json.load(file)

data = pd.read_csv('initial_data.csv')

# Загрузка модели 
with open('fitted_model.pkl', 'rb') as fd:
    pipeline = joblib.load(fd)

In [4]:
input_example = data[:10]
prediction = pipeline.predict(data[:10])
signature = mlflow.models.infer_signature(input_example, prediction)

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.log_metrics(metrics)
    
    model_info = mlflow.sklearn.log_model(
        pipeline, 
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        signature=signature,
        input_example=input_example,
        await_registration_for=60
		)

/home/mle-user/mle-project-sprint-2-v001/.venv_sprint/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Successfully registered model 'model_price_appartment'.
2025/02/25 13:39:30 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seco